In [1]:
import networkx as nx
import warnings
from itertools import islice
# import tqdm
# import matplotlib.pyplot as plt
# import pygraphviz as pgv
# from networkx.drawing.nx_agraph import graphviz_layout
from elasticsearch import Elasticsearch
from elasticsearch_dsl import connections

warnings.filterwarnings('ignore')


---

### `generate_netowrkx_graph()` 

`DiGraph`: holds directed edges. Self loops are allowed but **multiple (parallel) edges are not.**    
<br>  
`MultiDiGraph`: a **directed graph** class **that can store multi-edges**.  
* Multi-edges are multiple edges between two nodes.  
* Each edge can hold optional data or attributes.  
* A MultiDiGraph holds directed edges. Self loops are allowed.

Methods -

In [16]:
def get_all_via_scan(index=None, size=10000, query_data=None, scroll='1m'):
    from elasticsearch.helpers import scan
    from elasticsearch_dsl import connections

    es = connections.get_connection()

    if not index:
        index = "smartapi_metakg_docs_consolidated"

    # Make the initial scan request
    response = scan(es, query=query_data, index=index, size=size, scroll=scroll)

    for hit in response:
        yield hit


def generate_networkx_graph(index, graph_type="simple_digraph", edges=False, query_data=None):
    # # Create a new directed graph
    if graph_type == "simple_digraph":
        G = nx.DiGraph()
    elif graph_type == "multi_digraph":
        G = nx.MultiDiGraph()

    # Scroll through search results
    for hit in get_all_via_scan(size=1000, query_data=query_data, index=index): 
        # Extract subject, object, and predicate from hit
        _subject = hit['_source']['subject']
        _object = hit['_source']['object']
        _predicate = hit['_source']['predicate']
        
        # Add edge to graph, with predicate as an attribute
        if edges is True:
            G.add_edge(_subject, _object, predicate=_predicate)
        else:
            G.add_edge(_subject, _object)

    return G

def graph_inspection(G):
    # Get and print the number of nodes and edges
    num_nodes = G.number_of_nodes()
    num_edges = G.number_of_edges()
    graph_type = type(G)
    
    print(f"Graph type: {graph_type}")
    print(f"Number of nodes: {num_nodes}")
    print(f"Number of edges: {num_edges}\n")

    # Define the slice indices
    start_index = 55
    end_index = 60

    # Print a slice of nodes
    print("Nodes:")
    for node in islice(G.nodes(), start_index, end_index):
        print(node)

    # Print a slice of edges and their attributes
    print("\nEdges:")
    edges = list(G.edges(data=True))
    for source, target, edge_data in islice(edges, start_index, end_index):
        predicate = edge_data.get("predicate", "No predicate")
        print(f"Edge: {source} -> {target}, Predicate={predicate}")

Initialize Index -

In [17]:
es = Elasticsearch()
index = "smartapi_metakg_docs_consolidated"
connections.create_connection(hosts=['localhost'])


<Elasticsearch([{'host': 'localhost'}])>

Basic `digraph`, nodes only (graph1)

In [5]:
query_data = {
    'q': 'api.name:BTE'
}

In [6]:
G = generate_networkx_graph(index, query_data=query_data)
graph_inspection(G)

Graph type: <class 'networkx.classes.digraph.DiGraph'>
Number of nodes: 43
Number of edges: 665

Nodes:

Edges:
Edge: CellularComponent -> Procedure, Predicate=No predicate
Edge: CellularComponent -> PathologicalProcess, Predicate=No predicate
Edge: CellularComponent -> MolecularActivity, Predicate=No predicate
Edge: CellularComponent -> PhysiologicalProcess, Predicate=No predicate
Edge: GrossAnatomicalStructure -> Cell, Predicate=No predicate


Basic `digraph`, with edges (graph2)

In [7]:
G_edges_ = generate_networkx_graph(index, "simple_digraph", True, query_data=query_data)
graph_inspection(G_edges_)

Graph type: <class 'networkx.classes.digraph.DiGraph'>
Number of nodes: 43
Number of edges: 665

Nodes:

Edges:
Edge: CellularComponent -> Procedure, Predicate=location_of
Edge: CellularComponent -> PathologicalProcess, Predicate=disrupted_by
Edge: CellularComponent -> MolecularActivity, Predicate=location_of
Edge: CellularComponent -> PhysiologicalProcess, Predicate=affects
Edge: GrossAnatomicalStructure -> Cell, Predicate=has_part


`multi-digraph`, with edges (graph3)

In [8]:
G_edges = generate_networkx_graph(index, "multi_digraph", True, query_data=query_data)
graph_inspection(G_edges)

Graph type: <class 'networkx.classes.multidigraph.MultiDiGraph'>
Number of nodes: 43
Number of edges: 1827

Nodes:

Edges:
Edge: ChemicalEntity -> PhenotypicFeature, Predicate=related_to
Edge: ChemicalEntity -> PhenotypicFeature, Predicate=predisposes
Edge: ChemicalEntity -> Polypeptide, Predicate=affects
Edge: ChemicalEntity -> Polypeptide, Predicate=interacts_with
Edge: ChemicalEntity -> Polypeptide, Predicate=subclass_of


---

### `all_simple_paths(G, source, target, cutoff=None)`  
Generate all simple paths in the graph G from source to target. A simple path is a path with no repeated nodes.  
*Note*: Each pair of nodes can be connected by multiple directed edges, possibly in both directions.
When computing simple paths for a `MultiDiGraph`, the function will consider all possible directed edges between nodes. So, if there are multiple edges from node A to node B, the paths generated can be different based on which edge is considered in the path.  
<br>
The main difference is that with a `MultiDiGraph`, there can be multiple simple paths between two nodes that follow the exact same sequence of nodes but traverse different edges. This scenario is not possible in a simple `DiGraph`.  
<br>
** it's crucial to note that `nx.all_simple_paths()` still returns paths without cycles for both types of graphs. The difference is purely based on the number and directionality of edges between nodes in the graph structure.

<br>
<br>

#### `has_path(G, source, target)`
Returns True if G has a path from source to target.



In [9]:
# Define the function to find all simple paths and check if a path exsists
def return_all_simple_paths(G, _source, _target, _cutoff=2):
    # check for existence b/w paths -- recommended to use for large graphs
    if nx.has_path(G, _source, _target):
        return list(nx.all_simple_paths(G, source=_source, target=_target, cutoff=_cutoff))
    else:
        return []

def display_pathway_results(path_list, start_index, end_index, edges=False):
    # Iterate through a portion of the edges and view their attributes
    portion_of_paths = islice(path_list, start_index, end_index)
    
    # Iterate through the edges and view their attributes
    print(f"[INFO] Pathways for source:{_source} & target:{_target} with cutoff:{_cutoff}")
    print(f"[INFO] Total pathways extracted - {len(path_list)}\n")

    for i, path in enumerate(portion_of_paths):
        print(f"Path {i}:", " -> ".join(path))

        if edges:
            # Iterate through the edges in the path
            for i in range(len(path) - 1):
                source_node = path[i]
                target_node = path[i + 1]
                
                # Get edge attributes for the edge between source and target nodes
                edge_data = G_edges.get_edge_data(source_node, target_node)
                # if edge_data:
                #     for edge_key, edge_info in edge_data.items():
                #         predicate = edge_info.get("predicate", "No predicate")
                #         print(f"Edge from {source_node} to {target_node} (Key: {edge_key}): Predicate={predicate}")
                # else:
                #     print(f"No edge between {source_node} and {target_node}")

                # Extract 'predicate' values using a list comprehension
                predicate_values = [data['predicate'] for data in edge_data.values()]
                # Print the list of 'predicate' values
                print(f'\n * {source_node} -> {target_node} * \nEdges: {", ".join(predicate_values)}')

            print()
            print("-"* 80)
            print()

Set resusable testing variables

In [14]:
_source="ComplexMolecularMixture"
_target="DiseaseOrPhenotypicFeature"

# Define the range of edges you want to iterate through
start_index = 0  # Start index
end_index = 10 #int(G_edges.number_of_edges()/1000)  

Graph 1 - 

In [13]:
_cutoff=3
all_simple_paths_G = return_all_simple_paths(G, _source, _target, _cutoff)
display_pathway_results(all_simple_paths_G, 0, 5)

[INFO] Pathways for source:ComplexMolecularMixture & target:DiseaseOrPhenotypicFeature with cutoff:3
[INFO] Total pathways extracted - 83

Path 0: ComplexMolecularMixture -> Disease -> SmallMolecule -> DiseaseOrPhenotypicFeature
Path 1: ComplexMolecularMixture -> Disease -> ChemicalEntity -> DiseaseOrPhenotypicFeature
Path 2: ComplexMolecularMixture -> Disease -> MolecularMixture -> DiseaseOrPhenotypicFeature
Path 3: ComplexMolecularMixture -> Disease -> Drug -> DiseaseOrPhenotypicFeature
Path 4: ComplexMolecularMixture -> PhenotypicFeature -> SmallMolecule -> DiseaseOrPhenotypicFeature


In [29]:
_cutoff=5
all_simple_paths_G_edges_ = return_all_simple_paths(G_edges_, _source, _target, _cutoff)


In [30]:
display_pathway_results(all_simple_paths_G_edges_, 0, 5, edges=True)

[INFO] Pathways for source:ChemicalEntity & target:PlanetaryEntity with cutoff:5
[INFO] Total pathways extracted - 20762602

Path 0: ChemicalEntity -> PlanetaryEntity

 * ChemicalEntity -> PlanetaryEntity * 
Edges: negatively_regulates, participates_in, positively_regulates, actively_involved_in, actively_involves, acts_upstream_of_negative_effect, acts_upstream_of_or_within_negative_effect, acts_upstream_of_or_within, acts_upstream_of_or_within_positive_effect, acts_upstream_of, acts_upstream_of_positive_effect, affects, capable_of, enables, regulates, related_to, process_negatively_regulates_process, has_active_ingredient, primarily_composed_of, temporally_related_to, associated_with, process_regulates_process, similar_to, catalyzes, overlaps, affects_response_to, process_positively_regulates_process, composed_primarily_of, has_upstream_actor, exacerbates, increases_response_to, disrupts, is_output_of, affected_by, treats, is_substrate_of, part_of, has_capability, preceded_by, has_pa

Test `multidigraph`

In [14]:
_source="ComplexMolecularMixture"
_target="DiseaseOrPhenotypicFeature"

# Define the range of edges you want to iterate through
start_index = 0  # Start index
end_index = 10 #int(G_edges.number_of_edges()/1000)  

In [30]:
_cutoff=2
all_simple_paths_G_edges = return_all_simple_paths(G_edges, _source, _target)
display_pathway_results(all_simple_paths_G_edges, 0, 5)

[INFO] Pathways for source:ComplexMolecularMixture & target:DiseaseOrPhenotypicFeature with cutoff:2
[INFO] Total pathways extracted - 12

Path 0: ComplexMolecularMixture -> ChemicalEntity -> DiseaseOrPhenotypicFeature
Path 1: ComplexMolecularMixture -> ChemicalEntity -> DiseaseOrPhenotypicFeature
Path 2: ComplexMolecularMixture -> ChemicalEntity -> DiseaseOrPhenotypicFeature
Path 3: ComplexMolecularMixture -> ChemicalEntity -> DiseaseOrPhenotypicFeature
Path 4: ComplexMolecularMixture -> SmallMolecule -> DiseaseOrPhenotypicFeature


In [36]:
cutoff=3
all_simple_paths_G_edges = return_all_simple_paths(G_edges, _source, _target, cutoff)
display_pathway_results(all_simple_paths_G_edges, 0, 5)

[INFO] Pathways for source:ComplexMolecularMixture & target:DiseaseOrPhenotypicFeature with cutoff:5
[INFO] Total pathways extracted - 1678

Path 0: ComplexMolecularMixture -> Disease -> SmallMolecule -> DiseaseOrPhenotypicFeature
Path 1: ComplexMolecularMixture -> Disease -> SmallMolecule -> DiseaseOrPhenotypicFeature
Path 2: ComplexMolecularMixture -> Disease -> SmallMolecule -> DiseaseOrPhenotypicFeature
Path 3: ComplexMolecularMixture -> Disease -> SmallMolecule -> DiseaseOrPhenotypicFeature
Path 4: ComplexMolecularMixture -> Disease -> SmallMolecule -> DiseaseOrPhenotypicFeature


In [37]:
_cutoff=4
all_simple_paths_G_edges = return_all_simple_paths(G_edges, _source, _target, _cutoff)
display_pathway_results(all_simple_paths_G_edges, 0, 5)

[INFO] Pathways for source:ComplexMolecularMixture & target:DiseaseOrPhenotypicFeature with cutoff:4
[INFO] Total pathways extracted - 110962

Path 0: ComplexMolecularMixture -> Disease -> PhenotypicFeature -> SmallMolecule -> DiseaseOrPhenotypicFeature
Path 1: ComplexMolecularMixture -> Disease -> PhenotypicFeature -> SmallMolecule -> DiseaseOrPhenotypicFeature
Path 2: ComplexMolecularMixture -> Disease -> PhenotypicFeature -> SmallMolecule -> DiseaseOrPhenotypicFeature
Path 3: ComplexMolecularMixture -> Disease -> PhenotypicFeature -> SmallMolecule -> DiseaseOrPhenotypicFeature
Path 4: ComplexMolecularMixture -> Disease -> PhenotypicFeature -> SmallMolecule -> DiseaseOrPhenotypicFeature


In [15]:
_cutoff=5
all_simple_paths_G_edges = return_all_simple_paths(G_edges, _source, _target, _cutoff)
display_pathway_results(all_simple_paths_G_edges, 0, 5)

[INFO] Pathways for source:ComplexMolecularMixture & target:DiseaseOrPhenotypicFeature with cutoff:5
[INFO] Total pathways extracted - 7219384

Path 0: ComplexMolecularMixture -> Disease -> PhenotypicFeature -> SmallMolecule -> ChemicalEntity -> DiseaseOrPhenotypicFeature
Path 1: ComplexMolecularMixture -> Disease -> PhenotypicFeature -> SmallMolecule -> ChemicalEntity -> DiseaseOrPhenotypicFeature
Path 2: ComplexMolecularMixture -> Disease -> PhenotypicFeature -> SmallMolecule -> ChemicalEntity -> DiseaseOrPhenotypicFeature
Path 3: ComplexMolecularMixture -> Disease -> PhenotypicFeature -> SmallMolecule -> ChemicalEntity -> DiseaseOrPhenotypicFeature
Path 4: ComplexMolecularMixture -> Disease -> PhenotypicFeature -> SmallMolecule -> ChemicalEntity -> DiseaseOrPhenotypicFeature


In [17]:
display_pathway_results(all_simple_paths_G_edges, 0, 5, edges=True)

[INFO] Pathways for source:ComplexMolecularMixture & target:DiseaseOrPhenotypicFeature with cutoff:5
[INFO] Total pathways extracted - 7219384

Path 0: ComplexMolecularMixture -> Disease -> PhenotypicFeature -> SmallMolecule -> ChemicalEntity -> DiseaseOrPhenotypicFeature

 * ComplexMolecularMixture -> Disease * 
Edges: affects, positively_correlated_with, negatively_correlated_with, ameliorates, treats, contributes_to, correlated_with

 * Disease -> PhenotypicFeature * 
Edges: related_to, has_phenotype, associated_with, correlated_with, subclass_of, positively_correlated_with, negatively_correlated_with, causes, caused_by, has_predisposing_factor, coexists_with, has_manifestation, is_diagnosed_by

 * PhenotypicFeature -> SmallMolecule * 
Edges: related_to, positively_correlated_with, negatively_correlated_with, correlated_with, treated_by, caused_by, affected_by, contribution_from, has_predisposing_factor, disrupted_by, associated_with, prevented_by, adverse_event_of

 * SmallMolecule

Test `digraph`

In [23]:
_cutoff=5
all_simple_paths_G_edges_ = return_all_simple_paths(G_edges_, _source, _target, _cutoff)
display_pathway_results(all_simple_paths_G_edges_, 0, 5, edges=True)

[INFO] Pathways for source:ComplexMolecularMixture & target:DiseaseOrPhenotypicFeature with cutoff:5
[INFO] Total pathways extracted - 29063

Path 0: ComplexMolecularMixture -> Disease -> PhenotypicFeature -> SmallMolecule -> ChemicalEntity -> DiseaseOrPhenotypicFeature

 * ComplexMolecularMixture -> Disease * 
Edges: affects, positively_correlated_with, negatively_correlated_with, ameliorates, treats, contributes_to, correlated_with

 * Disease -> PhenotypicFeature * 
Edges: related_to, has_phenotype, associated_with, correlated_with, subclass_of, positively_correlated_with, negatively_correlated_with, causes, caused_by, has_predisposing_factor, coexists_with, has_manifestation, is_diagnosed_by

 * PhenotypicFeature -> SmallMolecule * 
Edges: related_to, positively_correlated_with, negatively_correlated_with, correlated_with, treated_by, caused_by, affected_by, contribution_from, has_predisposing_factor, disrupted_by, associated_with, prevented_by, adverse_event_of

 * SmallMolecule -

---  
Brainstorm -

In [31]:
# for path in all_simple_paths_G_edges_:
#     print(path)
edge_data = G_edges.get_edge_data(_source, _target)
print(edge_data)

# Extract 'predicate' values using a list comprehension
predicate_values = [data['predicate'] for data in edge_data.values()]
print()
# Print the list of 'predicate' values
print(predicate_values)

{0: {'predicate': 'negatively_regulates'}, 1: {'predicate': 'participates_in'}, 2: {'predicate': 'positively_regulates'}, 3: {'predicate': 'actively_involved_in'}, 4: {'predicate': 'actively_involves'}, 5: {'predicate': 'acts_upstream_of_negative_effect'}, 6: {'predicate': 'acts_upstream_of_or_within_negative_effect'}, 7: {'predicate': 'acts_upstream_of_or_within'}, 8: {'predicate': 'acts_upstream_of_or_within_positive_effect'}, 9: {'predicate': 'acts_upstream_of'}, 10: {'predicate': 'acts_upstream_of_positive_effect'}, 11: {'predicate': 'affects'}, 12: {'predicate': 'capable_of'}, 13: {'predicate': 'enables'}, 14: {'predicate': 'regulates'}, 15: {'predicate': 'related_to'}, 16: {'predicate': 'process_negatively_regulates_process'}, 17: {'predicate': 'has_active_ingredient'}, 18: {'predicate': 'primarily_composed_of'}, 19: {'predicate': 'temporally_related_to'}, 20: {'predicate': 'associated_with'}, 21: {'predicate': 'process_regulates_process'}, 22: {'predicate': 'similar_to'}, 23: {'

---